In [ ]:
# Task 1

import numpy as np
import pywt
import matplotlib.pyplot as plt
import cv2

def load_and_preprocess_image(image_path):
    """
    Load image in grayscale and normalize
    """
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError("Could not load image")
    return image

def perform_wavelet_transform(image, wavelet='haar', levels=2):
    """
    Perform multi-level 2D Discrete Wavelet Transform
    """
    coeffs = pywt.wavedec2(image, wavelet, level=levels)
    return coeffs

def plot_wavelet_components(coeffs):
    """
    Visualize wavelet transform components
    """
    plt.figure(figsize=(15, 10))
    
    # Approximation (LL)
    plt.subplot(2, 3, 1)
    plt.title('Approximation Coefficients (LL)')
    plt.imshow(coeffs[0], cmap='gray')
    
    # Details for each level
    titles = ['Level 1 Details', 'Level 2 Details']
    positions = [2, 4]
    
    for level, (detail_coeffs, pos) in enumerate(zip(coeffs[1:], positions), 1):
        # LH, HL, HH for each level
        detail_types = ['LH', 'HL', 'HH']
        
        for i, (detail_type, coeff) in enumerate(zip(detail_types, detail_coeffs)):
            plt.subplot(2, 3, pos + i)
            plt.title(f'Level {level} {detail_type}')
            plt.imshow(coeff, cmap='gray')
    
    plt.tight_layout()
    plt.show()

def main():
    image_path = 'mri.png'
    
    image = load_and_preprocess_image(image_path)
    
    # Perform wavelet transform
    coeffs = perform_wavelet_transform(image)
    
    # Plot components
    plot_wavelet_components(coeffs)

  

if __name__ == '__main__':
    main()


    # Task 2

import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt

def load_image(image_path):
    """
    Load grayscale image and ensure even dimensions
    """
    # Read image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        raise ValueError(f"Unable to load image: {image_path}")
    
    # Ensure even dimensions
    height, width = image.shape
    height = height if height % 2 == 0 else height - 1
    width = width if width % 2 == 0 else width - 1
    
    return image[:height, :width]

def compress_image_dwt(image, threshold_percent=10):
    """
    Compress image using Discrete Wavelet Transform
    
    Parameters:
    - image: Input grayscale image
    - threshold_percent: Percentage of max coefficient to threshold
    
    Returns:
    - Compressed reconstructed image
    - Compression metrics
    """
    # Perform wavelet decomposition
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    
    # Create compressed coefficients
    compressed_coeffs = [coeffs[0]]  # Approximation coefficients
    
    # Threshold detail coefficients
    for detail_tuple in coeffs[1:]:
        compressed_details = []
        for detail in detail_tuple:
            # Calculate threshold
            threshold = np.max(np.abs(detail)) * (threshold_percent / 100)
            
            # Apply thresholding
            compressed_detail = detail.copy()
            compressed_detail[np.abs(compressed_detail) < threshold] = 0
            compressed_details.append(compressed_detail)
        
        compressed_coeffs.append(tuple(compressed_details))
    
    # Reconstruct image
    reconstructed = pywt.waverec2(compressed_coeffs, 'haar')
    
    # Clip and convert to uint8
    reconstructed = np.clip(reconstructed, 0, 255).astype(np.uint8)
    
    # Ensure same dimensions as original
    reconstructed = reconstructed[:image.shape[0], :image.shape[1]]
    
    # Calculate compression metrics
    compression_ratio = calculate_compression_ratio(image, reconstructed)
    mse = calculate_mse(image, reconstructed)
    psnr = calculate_psnr(image, reconstructed)
    
    return reconstructed, compression_ratio, mse, psnr

def calculate_compression_ratio(original, compressed):
    """Calculate compression ratio"""
    return original.nbytes / compressed.nbytes

def calculate_mse(original, reconstructed):
    """Calculate Mean Squared Error"""
    return np.mean((original.astype(float) - reconstructed.astype(float)) ** 2)

def calculate_psnr(original, reconstructed):
    """Calculate Peak Signal-to-Noise Ratio"""
    mse = calculate_mse(original, reconstructed)
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse)) if mse > 0 else float('inf')

def visualize_compression(original, reconstructed):
    """Visualize original and compressed images"""
    plt.figure(figsize=(12, 4))
    
    plt.subplot(131)
    plt.title('Original Image')
    plt.imshow(original, cmap='gray')
    plt.axis('off')
    
    plt.subplot(132)
    plt.title('Compressed Image')
    plt.imshow(reconstructed, cmap='gray')
    plt.axis('off')
    
    plt.subplot(133)
    plt.title('Difference Image')
    difference = np.abs(original - reconstructed)
    plt.imshow(difference, cmap='gray')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def main():
    # Replace with your MRI image path
    image_path = 'mri.png'
    
    try:
        # Load image
        image = load_image(image_path)
        
        # Compress image
        reconstructed, cr, mse, psnr = compress_image_dwt(image)
        
        # Visualize results
        visualize_compression(image, reconstructed)
        
        # Print compression metrics
        print("DWT Compression Results:")
        print(f"Compression Ratio: {cr:.2f}")
        print(f"Mean Squared Error: {mse:.2f}")
        print(f"Peak Signal-to-Noise Ratio: {psnr:.2f} dB")
        
        # Interpretation
        print("\nCompression Impact Analysis:")
        print("- Wavelet Transform preserves key image features")
        print("- Thresholding removes less significant coefficients")
        print("- Trade-off between compression and image quality")
        
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()


# Task 3

import numpy as np
import cv2
import pywt
import matplotlib.pyplot as plt
import time

def load_image(image_path):
    """
    Load grayscale image and ensure even dimensions
    """
    # Read image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is None:
        raise ValueError(f"Unable to load image: {image_path}")
    
    # Ensure even dimensions
    height, width = image.shape
    height = height if height % 2 == 0 else height - 1
    width = width if width % 2 == 0 else width - 1
    
    return image[:height, :width]

def compress_with_dwt(image, threshold_percent=10):
    """Compress using Discrete Wavelet Transform"""
    start_time = time.time()
    
    # Wavelet Transform
    coeffs = pywt.wavedec2(image, 'haar', level=2)
    
    # Create compressed coefficients
    compressed_coeffs = [coeffs[0]]
    
    # Threshold detail coefficients
    for detail_tuple in coeffs[1:]:
        compressed_details = []
        for detail in detail_tuple:
            threshold = np.max(np.abs(detail)) * (threshold_percent / 100)
            compressed_detail = detail.copy()
            compressed_detail[np.abs(compressed_detail) < threshold] = 0
            compressed_details.append(compressed_detail)
        
        compressed_coeffs.append(tuple(compressed_details))
    
    # Reconstruct image
    reconstructed = pywt.waverec2(compressed_coeffs, 'haar')
    reconstructed = np.clip(reconstructed, 0, 255).astype(np.uint8)
    reconstructed = reconstructed[:image.shape[0], :image.shape[1]]
    
    dwt_time = time.time() - start_time
    
    return reconstructed, dwt_time

def compress_with_fft(image, threshold_percent=10):
    """Compress using Fast Fourier Transform"""
    start_time = time.time()
    
    # Compute 2D FFT
    f_transform = np.fft.fft2(image)
    fshift = np.fft.fftshift(f_transform)
    
    # Compute magnitude spectrum
    magnitude_spectrum = 20 * np.log10(np.abs(fshift) + 1)
    
    # Compute threshold
    threshold = np.max(magnitude_spectrum) * (threshold_percent / 100)
    
    # Zero out low magnitude components
    fshift[magnitude_spectrum < threshold] = 0
    
    # Inverse FFT
    f_ishift = np.fft.ifftshift(fshift)
    reconstructed = np.abs(np.fft.ifft2(f_ishift))
    reconstructed = np.clip(reconstructed, 0, 255).astype(np.uint8)
    reconstructed = reconstructed[:image.shape[0], :image.shape[1]]
    
    fft_time = time.time() - start_time
    
    return reconstructed, fft_time

def calculate_metrics(original, reconstructed):
    """Calculate compression metrics"""
    # Compression Ratio
    compression_ratio = original.nbytes / reconstructed.nbytes
    
    # Mean Squared Error
    mse = np.mean((original.astype(float) - reconstructed.astype(float)) ** 2)
    
    # Peak Signal-to-Noise Ratio
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse)) if mse > 0 else float('inf')
    
    return compression_ratio, mse, psnr

def visualize_comparison(original, dwt_reconstructed, fft_reconstructed):
    """Visualize comparison of original and reconstructed images"""
    plt.figure(figsize=(15, 5))
    
    # Original Image
    plt.subplot(231)
    plt.title('Original Image')
    plt.imshow(original, cmap='gray')
    plt.axis('off')
    
    # DWT Reconstructed
    plt.subplot(232)
    plt.title('DWT Reconstructed')
    plt.imshow(dwt_reconstructed, cmap='gray')
    plt.axis('off')
    
    # FFT Reconstructed
    plt.subplot(233)
    plt.title('FFT Reconstructed')
    plt.imshow(fft_reconstructed, cmap='gray')
    plt.axis('off')
    
    # DWT Difference
    plt.subplot(234)
    plt.title('DWT Difference')
    dwt_diff = np.abs(original - dwt_reconstructed)
    plt.imshow(dwt_diff, cmap='gray')
    plt.axis('off')
    
    # FFT Difference
    plt.subplot(235)
    plt.title('FFT Difference')
    fft_diff = np.abs(original - fft_reconstructed)
    plt.imshow(fft_diff, cmap='gray')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

def main():
    # Replace with your MRI image path
    image_path = 'mri.png'
    
    try:
        # Load image
        image = load_image(image_path)
        
        # DWT Compression
        dwt_reconstructed, dwt_time = compress_with_dwt(image)
        dwt_cr, dwt_mse, dwt_psnr = calculate_metrics(image, dwt_reconstructed)
        
        # FFT Compression
        fft_reconstructed, fft_time = compress_with_fft(image)
        fft_cr, fft_mse, fft_psnr = calculate_metrics(image, fft_reconstructed)
        
        # Visualize Results
        visualize_comparison(image, dwt_reconstructed, fft_reconstructed)
        
        # Print Comparison Results
        print("Compression Techniques Comparison:")
        print("\nDWT Compression Metrics:")
        print(f"Compression Ratio: {dwt_cr:.2f}")
        print(f"Mean Squared Error: {dwt_mse:.2f}")
        print(f"Peak Signal-to-Noise Ratio: {dwt_psnr:.2f} dB")
        print(f"Computation Time: {dwt_time:.4f} seconds")
        
        print("\nFFT Compression Metrics:")
        print(f"Compression Ratio: {fft_cr:.2f}")
        print(f"Mean Squared Error: {fft_mse:.2f}")
        print(f"Peak Signal-to-Noise Ratio: {fft_psnr:.2f} dB")
        print(f"Computation Time: {fft_time:.4f} seconds")
        
        print("\nAnalysis:")
        print("1. Feature Retention:")
        print("   - DWT preserves spatial and frequency information")
        print("   - FFT loses spatial localization")
        
        print("\n2. Compression Efficiency:")
        print(f"   - DWT Compression Ratio: {dwt_cr:.2f}")
        print(f"   - FFT Compression Ratio: {fft_cr:.2f}")
        
        print("\n3. Image Quality:")
        print(f"   - DWT PSNR: {dwt_psnr:.2f} dB")
        print(f"   - FFT PSNR: {fft_psnr:.2f} dB")
        
        print("\nConclusion:")
        print("Recommended for Medical Imaging: Discrete Wavelet Transform (DWT)")
        print("- Better feature preservation")
        print("- More precise edge and texture representation")
        
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()